## Solving Hangman with Statistical Letter Prediction

This project aims to build a Hangman player that can guess hidden words with high accuracy using positional information.
We begin by counting how many substrings occur at each position in the word, considering both forward (from the beginning) and backward (from the end) directions. The goal of utilizing these statistics is to capture common **prefix** and **suffix** patterns.

We combine scores from five different models—positional frequency, bigram, trigram, four-gram, and five-gram—into a single aggregated score, which is then used to determine the next letter guess. We next describe these five scoring schemes:

### **Positional Score**
We identify candidate words that match the current masked pattern, considering both prefix and suffix alignment. For each masked position, we compute a score for every letter based on how frequently it appears in that position across the matched words. These scores are then aggregated over all masked positions to compute a total score for each candidate letter.

### **Bigram Score**
For each masked position that is adjacent to a revealed letter, we consult bigram frequency statistics. We determine how often each candidate letter appears in that specific context—both when looking from the start and from the end of the word. Scores are aggregated across all such bigram contexts.

### **Trigram Score**
Similar to bigrams, we look at masked positions that are part of a substring containing two revealed neighboring letters. Using trigram frequency statistics, we score each letter based on how often it appears in that specific three-letter context. The final trigram score is computed by aggregating across all relevant positions.

### **Fourgram and Fivegram Score**
The same process is extended to four-gram and five-gram contexts.


After computing all n-gram-based scores, we combine them using a **weighted sum**:
- Higher-order n-grams (e.g., five-gram) are given **greater weight**
- Lower-order n-grams (e.g., bigram) receive **less weight**
- The **positional score** is also assigned a relatively high weight due to its strong predictive power

---

### Accuracy
This combined scoring strategy achieved an accuracy of approximately **55–60%** depending on the held-out evaluation set.


In [7]:
import collections
import re
import random

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### `LocalHangmanSimulator` Class

The `LocalHangmanSimulator` class is responsible for evaluating the performance of a letter-guessing strategy in a simulated game of Hangman. It takes a list of test words and a guesser object, then repeatedly plays games to estimate win rates.

In [5]:
class LocalHangmanSimulator:
    def __init__(self, held_out_words, guess_function, verbose=False):
        """
        Initialize the Hangman simulator.

        Parameters:
        - held_out_words: List[str]
            A list of words to use as test cases (disjoint from training data).
        - guess_function: object
            An object that has .reset() and .guess(masked_str) methods.
        - verbose: bool
            If True, prints each guess and the current masked word state for debugging/visualization.
        """
        self.held_out_words = held_out_words
        self.guess_function = guess_function
        self.verbose = verbose

    def play_one_game(self):
        """
        Simulate one game of Hangman using a word selected at random from the held-out list.

        Returns:
        - success: bool
            True if the word was successfully guessed within the allowed number of incorrect guesses, False otherwise.
        """
        secret_word = random.choice(self.held_out_words)  # Choose a word to guess
        guessed_letters = set()                           # Track guessed letters to avoid repetition
        incorrect_guesses = 0
        max_incorrect = 6                                 # Hangman limit
        current_mask = ['_'] * len(secret_word)           # Initialize the masked word

        self.guess_function.reset()                       # Reset the guesser’s state

        while incorrect_guesses < max_incorrect and '_' in current_mask:
            masked_str = ' '.join(current_mask)
            guess = self.guess_function.guess(masked_str) # Get a letter guess from the guesser

            if self.verbose:
                print(f"Guessed letter: {guess}")

            if guess in guessed_letters:                  # Skip already guessed letters
                continue
            guessed_letters.add(guess)

            if guess in secret_word:
                # Reveal the correctly guessed letter in all matching positions
                for i, char in enumerate(secret_word):
                    if char == guess:
                        current_mask[i] = guess
            else:
                incorrect_guesses += 1                    # Count wrong guesses

            if self.verbose:
                print(f"Current mask: {' '.join(current_mask)}")

        return '_' not in current_mask                    # Win if no blanks are left

    def evaluate(self, games=1000):
        """
        Run multiple Hangman games and compute the win rate.

        Parameters:
        - games: int
            Number of games to simulate.

        Returns:
        - win_rate: float
            The proportion of games successfully won.
        """
        wins = 0
        for i in range(1, games + 1):
            if self.play_one_game():
                wins += 1
        return wins / games


### `LocalGuesserPositionUnigram` Class

The `LocalGuesserPositionUnigram` class implements a statistical letter-guessing strategy for Hangman. It combines frequency-based positional scoring with multiple n-gram models (from unigram to five-gram), using both forward and reverse positional patterns.


In [8]:

class LocalGuesserPositionUnigram:
    def __init__(self, dictionary_subset):

        self.full_dictionary = dictionary_subset #training set
        self.guessed_letters = []
        self.letter_set = set() #Alphabet

        #Compute the list of letters sorted by most common to least common
        fallback_counter = collections.Counter()
        for word in self.full_dictionary:
            fallback_counter.update(set(word))
            self.sorted_letters = [letter for letter, _ in fallback_counter.most_common()]

        self.positional_bigram_counter = collections.defaultdict(collections.Counter)
        self.reverse_positional_bigram_counter = collections.defaultdict(collections.Counter)
        self.positional_trigram_counter = collections.defaultdict(collections.Counter)
        self.reverse_positional_trigram_counter = collections.defaultdict(collections.Counter)
        self.positional_fourgram_counter = collections.defaultdict(collections.Counter)
        self.reverse_positional_fourgram_counter = collections.defaultdict(collections.Counter)
        self.positional_fivegram_counter = collections.defaultdict(collections.Counter)
        self.reverse_positional_fivegram_counter = collections.defaultdict(collections.Counter)


        # --- Build the positional n-gram models (forward and reverse directions) ---
        for word in self.full_dictionary:
            L = len(word)
            self.letter_set.update(word)

             # --- Build bigram (2-gram) counts ---
            for i in range(L - 1):
                bigram = (word[i], word[i+1])
                self.positional_bigram_counter[i][bigram] += 1
                self.reverse_positional_bigram_counter[L - 2 - i][bigram] += 1

            # --- Build trigram (3-gram) counts ---
            for i in range(L - 2):
                trigram = (word[i], word[i+1], word[i+2])
                self.positional_trigram_counter[i][trigram] += 1
                self.reverse_positional_trigram_counter[L - 3 - i][trigram] += 1

            # --- Build fourgram (4-gram) counts ---
            for i in range(L - 3):
              fourgram = (word[i], word[i+1], word[i+2], word[i+3])
              self.positional_fourgram_counter[i][fourgram] += 1
              self.reverse_positional_fourgram_counter[L - 4 - i][fourgram] += 1

            # --- Build fivegram (5-gram) counts ---
            for i in range(L-4):
                fivegram = (word[i], word[i+1], word[i+2], word[i+3], word[i+4])
                self.positional_fivegram_counter[i][fivegram] += 1
                self.reverse_positional_fivegram_counter[L-5-i][fivegram] += 1


    def reset(self):
        self.guessed_letters = []

    def matches_clean_word(self, w, clean_word):
      if len(w) != len(clean_word):
          return False
      for wc, cc in zip(w, clean_word):
          if cc != '.' and wc != cc:
              return False
      return True

    def get_compatible_words(self, clean_word):
        candidates = [w for w in self.full_dictionary]

        compatible_start= []
        for w in candidates:
            prefix = w[:len(clean_word)]
             # Check if the prefix matches the clean_word
            if self.matches_clean_word(prefix, clean_word) and all(
                l not in self.guessed_letters for l, c in zip(prefix, clean_word) if c == '.'
            ):
                compatible_start.append(w)

        # Find words that match the clean_word at the end
        compatible_end = []
        for w in candidates:
            suffix = w[-len(clean_word):]
             # Check if the suffix matches the clean_word
            if self.matches_clean_word(suffix, clean_word) and all(
                l not in self.guessed_letters for l, c in zip(suffix, clean_word) if c == '.'
            ):
                compatible_end.append(w)

        return compatible_start, compatible_end


    def positional_unigram_score(self, word):
        # Preprocess the word
        clean_word = word[::2].replace("_", ".")
        # Find compatible words matching the pattern at the start and end
        compatible_start,compatible_end = self.get_compatible_words(clean_word)

        if not compatible_start and not compatible_end:
            return collections.Counter()

        L = len(clean_word)
        start_counters = [collections.Counter() for _ in range(L)]
        end_counters = [collections.Counter() for _ in range(L)]

        # --- Collect counts from start matches ---
        for w in compatible_start:
            for idx, ch in enumerate(clean_word):
                if idx < len(w) and ch == '.':
                    start_counters[idx][w[idx]] += 1

        # --- Collect counts from end matches ---
        for w in compatible_end:
            len_w = len(w)
            for idx, ch in enumerate(clean_word):
                rel_idx = idx - L  # Relative index from the end
                if abs(rel_idx) >= len_w:
                    continue
                if ch == '.':
                    end_counters[idx][w[rel_idx]] += 1

        scores = collections.Counter()

        for idx in range(L):
            total_start = sum(start_counters[idx].values())
            total_end = sum(end_counters[idx].values())

            for letter, count in start_counters[idx].items():
                if letter not in self.guessed_letters and total_start > 0:
                    scores[letter] += (count / total_start)

            for letter, count in end_counters[idx].items():
                if letter not in self.guessed_letters and total_end > 0:
                    scores[letter] += (count / total_end)

        # --- Normalization ---
        total = sum(scores.values())
        if total > 0:
            for letter in scores:
                scores[letter] /= total

        return scores

    def bigram_scores(self, word):
        # Preprocess
        clean_word = word[::2].replace("_", ".")

        L = len(clean_word)
        scores = collections.Counter()

        for idx, ch in enumerate(clean_word):
            if ch != '.':
                continue

            # Neighboring characters
            prev = clean_word[idx-1] if idx >= 1 else None
            next = clean_word[idx+1] if idx < L-1 else None

            start_pos = idx
            end_pos = (L-1) - idx

            for candidate in self.letter_set:
                # Skip already guessed letters
                if candidate in self.guessed_letters:
                    continue
                score = 0
                # If there is a known previous letter, use (prev, candidate) bigram statistics
                if prev and prev != '.':
                    score += self.positional_bigram_counter[start_pos-1][(prev, candidate)]
                    score += self.reverse_positional_bigram_counter[end_pos][(prev, candidate)]
                # If there is a known next letter, use (candidate, next) bigram statistics
                if next and next != '.':
                    score += self.positional_bigram_counter[start_pos][(candidate, next)]
                    score += self.reverse_positional_bigram_counter[end_pos-1][(candidate, next)]

                scores[candidate] += score

        # --- Normalization ---
        total = sum(scores.values())
        if total > 0:
            for letter in scores:
                scores[letter] /= total

        return scores

    def trigram_scores(self, word):
        # Preprocess
        clean_word = word[::2].replace("_", ".")

        L = len(clean_word)
        scores = collections.Counter()

        for idx, ch in enumerate(clean_word):
            if ch != '.':
                continue
            # Get neighboring characters: two before and two after
            prev2 = clean_word[idx-2] if idx >= 2 else None
            prev1 = clean_word[idx-1] if idx >= 1 else None
            next1 = clean_word[idx+1] if idx < L-1 else None
            next2 = clean_word[idx+2] if idx < L-2 else None

            start_pos = idx
            end_pos = (L-1) - idx

            for candidate in self.letter_set:
                if candidate in self.guessed_letters:
                    continue

                score = 0

                # --- Case 1: Two letters before + candidate ---
                if prev2 and prev2 != '.' and prev1 and prev1 != '.':
                    score += 0.5 * self.positional_trigram_counter[start_pos-2][(prev2, prev1, candidate)]
                    score += 0.5 * self.reverse_positional_trigram_counter[end_pos][(prev2, prev1, candidate)]

                # --- Case 2: One letter before + candidate + one after ---
                if prev1 and prev1 != '.' and next1 and next1 != '.':
                    score += 0.5 * self.positional_trigram_counter[start_pos-1][(prev1, candidate, next1)]
                    score += 0.5 * self.reverse_positional_trigram_counter[end_pos-1][(prev1, candidate, next1)]

                # --- Case 3: Candidate + two letters after ---
                if next1 and next1 != '.' and next2 and next2 != '.':
                    score += 0.5 * self.positional_trigram_counter[start_pos][(candidate, next1, next2)]
                    score += 0.5 * self.reverse_positional_trigram_counter[end_pos-2][(candidate, next1, next2)]

                scores[candidate] += score

        # --- Normalization ---
        total = sum(scores.values())
        if total > 0:
            for letter in scores:
                scores[letter] /= total

        return scores


    def fourgram_scores(self, word):
      # Preprocess
      clean_word = word[::2].replace("_", ".")
      L = len(clean_word)
      scores = collections.Counter()

      for idx, ch in enumerate(clean_word):
          if ch != '.':
              continue

          # Get neighboring characters: up to three before and three after
          prev3 = clean_word[idx-3] if idx >= 3 else None
          prev2 = clean_word[idx-2] if idx >= 2 else None
          prev1 = clean_word[idx-1] if idx >= 1 else None
          next1 = clean_word[idx+1] if idx < L-1 else None
          next2 = clean_word[idx+2] if idx < L-2 else None
          next3 = clean_word[idx+3] if idx < L-3 else None

          start_pos = idx
          end_pos = (L-1) - idx

          for candidate in self.letter_set:
              if candidate in self.guessed_letters:
                  continue

              score = 0

              # --- Case 1: three letters before + candidate ---
              if prev3 and prev3 != '.' and prev2 and prev2 != '.' and prev1 and prev1 != '.':
                  score += 0.5 * self.positional_fourgram_counter[start_pos-3][(prev3, prev2, prev1, candidate)]
                  score += 0.5 * self.reverse_positional_fourgram_counter[end_pos][(prev3, prev2, prev1, candidate)]

              # --- Case 2: two before + candidate + one after ---
              if prev2 and prev2 != '.' and prev1 and prev1 != '.' and next1 and next1 != '.':
                  score += 0.5 * self.positional_fourgram_counter[start_pos-2][(prev2, prev1, candidate, next1)]
                  score += 0.5 * self.reverse_positional_fourgram_counter[end_pos-1][(prev2, prev1, candidate, next1)]

              # --- Case 3: one before + candidate + two after ---
              if prev1 and prev1 != '.' and next1 and next1 != '.' and next2 and next2 != '.':
                  score += 0.5 * self.positional_fourgram_counter[start_pos-1][(prev1, candidate, next1, next2)]
                  score += 0.5 * self.reverse_positional_fourgram_counter[end_pos-2][(prev1, candidate, next1, next2)]

              # --- Case 4: candidate + three letters after ---
              if next1 and next1 != '.' and next2 and next2 != '.' and next3 and next3 != '.':
                  score += 0.5 * self.positional_fourgram_counter[start_pos][(candidate, next1, next2, next3)]
                  score += 0.5 * self.reverse_positional_fourgram_counter[end_pos-3][(candidate, next1, next2, next3)]

              scores[candidate] += score

      # --- Normalization ---
      total = sum(scores.values())
      if total > 0:
          for letter in scores:
              scores[letter] /= total

      return scores

    def fivegram_scores(self, word):
      # Preprocess
      clean_word = word[::2].replace("_", ".")
      L = len(clean_word)
      scores = collections.Counter()

      for idx, ch in enumerate(clean_word):
          if ch != '.':
              continue

          # Get neighboring characters: up to four before and four after
          prev4 = clean_word[idx-4] if idx >= 4 else None
          prev3 = clean_word[idx-3] if idx >= 3 else None
          prev2 = clean_word[idx-2] if idx >= 2 else None
          prev1 = clean_word[idx-1] if idx >= 1 else None
          next1 = clean_word[idx+1] if idx < L-1 else None
          next2 = clean_word[idx+2] if idx < L-2 else None
          next3 = clean_word[idx+3] if idx < L-3 else None
          next4 = clean_word[idx+4] if idx < L-4 else None

          start_pos = idx
          end_pos = (L-1) - idx

          for candidate in self.letter_set:
              if candidate in self.guessed_letters:
                  continue

              score = 0

              # --- Case 1: four before + candidate ---
              if prev4 and prev4 != '.' and prev3 and prev3 != '.' and prev2 and prev2 != '.' and prev1 and prev1 != '.':
                  score += 0.5 * self.positional_fivegram_counter[start_pos-4][(prev4, prev3, prev2, prev1, candidate)]
                  score += 0.5 * self.reverse_positional_fivegram_counter[end_pos][(prev4, prev3, prev2, prev1, candidate)]

              # --- Case 2: three before + candidate + one after ---
              if prev3 and prev3 != '.' and prev2 and prev2 != '.' and prev1 and prev1 != '.' and next1 and next1 != '.':
                  score += 0.5 * self.positional_fivegram_counter[start_pos-3][(prev3, prev2, prev1, candidate, next1)]
                  score += 0.5 * self.reverse_positional_fivegram_counter[end_pos-1][(prev3, prev2, prev1, candidate, next1)]

              # --- Case 3: two before + candidate + two after ---
              if prev2 and prev2 != '.' and prev1 and prev1 != '.' and next1 and next1 != '.' and next2 and next2 != '.':
                  score += 0.5 * self.positional_fivegram_counter[start_pos-2][(prev2, prev1, candidate, next1, next2)]
                  score += 0.5 * self.reverse_positional_fivegram_counter[end_pos-2][(prev2, prev1, candidate, next1, next2)]

              # --- Case 4: one before + candidate + three after ---
              if prev1 and prev1 != '.' and next1 and next1 != '.' and next2 and next2 != '.' and next3 and next3 != '.':
                  score += 0.5 * self.positional_fivegram_counter[start_pos-1][(prev1, candidate, next1, next2, next3)]
                  score += 0.5 * self.reverse_positional_fivegram_counter[end_pos-3][(prev1, candidate, next1, next2, next3)]

              # --- Case 5: candidate + four after ---
              if next1 and next1 != '.' and next2 and next2 != '.' and next3 and next3 != '.' and next4 and next4 != '.':
                  score += 0.5 * self.positional_fivegram_counter[start_pos][(candidate, next1, next2, next3, next4)]
                  score += 0.5 * self.reverse_positional_fivegram_counter[end_pos-4][(candidate, next1, next2, next3, next4)]

              scores[candidate] += score

      # --- Normalization ---
      total = sum(scores.values())
      if total > 0:
          for letter in scores:
              scores[letter] /= total

      return scores


    def scale_counter(self, counter, scalar):
        return collections.Counter({k: v * scalar for k, v in counter.items()})


    def guess(self, word):

        # Compute the scores from each model
        positional_score = self.positional_unigram_score(word)
        bigram_score = self.bigram_scores(word)
        trigram_score = self.trigram_scores(word)
        fourgram_score = self.fourgram_scores(word)
        fivegram_score = self.fivegram_scores(word)

        # Combine the scores using predefined weights
        letter_scores = (
            self.scale_counter(bigram_score, 0.6) +
            self.scale_counter(positional_score, 3.1) +
            self.scale_counter(trigram_score, 1.4) +
            self.scale_counter(fourgram_score, 2.1) +
            self.scale_counter(fivegram_score, 2.4)
        )

        # Choose the most common letter if no scores are found
        if not letter_scores:
            for letter in self.sorted_letters:
                if letter not in self.guessed_letters:
                    self.guessed_letters.append(letter)
                    return letter

        # Pick the best letter with the highest combined score
        best_letter = letter_scores.most_common(1)[0][0]
        self.guessed_letters.append(best_letter)

        return best_letter


### Model Evaluation

This section loads a large word list, splits it into training and evaluation sets, and tests the performance of the `LocalGuesserPositionUnigram` strategy using the `LocalHangmanSimulator`. Over 1,000 simulated Hangman games, the model achieves a win rate in the range of **55–60%**, depending on the random sampling of the holdout set.



In [12]:
# Load the word list from file (250,000 training words)
with open("/content/drive/MyDrive/Hangman/local/words_250000_train.txt", 'r') as f:
    all_words = [line.strip() for line in f]  # Strip newline characters

# Randomize the word order
random.shuffle(all_words)

# Split the word list into training and holdout sets
training_set = all_words[1000:]     # Use 249,000 words to train the guesser
holdout_set = all_words[:1000]      # Use 1,000 unseen words for evaluation

# Create an instance of a position-aware unigram-based guesser
guesser = LocalGuesserPositionUnigram(dictionary_subset=training_set)

# Initialize the Hangman simulator with the holdout words and guesser
simulator = LocalHangmanSimulator(held_out_words=holdout_set, guess_function=guesser, verbose=False)

# Run 1,000 simulated Hangman games and compute win rate
win_rate =simulator.evaluate(games=1000)

# Print the win rate
print(f"Win rate over 1000 games: {win_rate * 100:.2f}%")

# Debug a single game with verbose output:
# guesser = LocalGuesserPositionUnigram(dictionary_subset=training_set, verbose=True)
# simulator = LocalHangmanSimulator(held_out_words=holdout_set, guess_function=guesser, verbose=True)
# simulator.evaluate(games=1)


Win rate over 1000 games: 60.70%
